# Data Cleaning 

This notebook will subset the columns needed and input missing data we will also one hot encode our categorical variables.

In [2]:
# data manipulation and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging 

# for saving the pipeline
import joblib

# from Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer

# from feature-engine
from feature_engine.imputation import (
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer,
)

from feature_engine.encoding import (
    RareLabelEncoder,
    OrdinalEncoder,
    OneHotEncoder
)

from feature_engine.transformation import (
    LogTransformer,
    YeoJohnsonTransformer,
)

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = None

logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

In [3]:
df_columns = ['id',
              'date',
             'price',
             'postcode',
             'type',
             'new_build',
             'land',
             'primary_address',
             'secondary_address',
             'street',
             'latitude',
             'longitude',
             'grid_ref',
             'county',
             'district',
             'ward',
             'constituency',
             'region',
             'middle_layer_super_output_area',
             'postcode_area',
             'postcode_district',
             'nearest_station',
             'distance_to_station',
             'police_force',
             'water_company',
             'average_income',
             'sewage_company',
             'travel_to_work_area',
             'rural_urban',
             'altitude',
             'region_name',
             'area_code',
             'adjusted_price']

In [40]:
# load dataset
df = pd.read_csv('../data/processed/pp_nottinghamshire.csv')

In [41]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['quarter'] = pd.DatetimeIndex(df['date']).quarter

In [42]:
df.to_csv('../data/processed/pp_nottinghamshire.csv', index=False)

In [6]:
random_seed = 42
# Split the data - train, validation and test
train_set, test_set = train_test_split(df,
                                       test_size=0.30,
                                       random_state=random_seed)

test_set, validation_set = train_test_split(test_set,
                                            test_size=0.20,
                                            random_state=random_seed)

logging.info(f"Training shape: {train_set.shape}")
logging.info(f"Validation shape: {validation_set.shape}")
logging.info(f"Test shape: {test_set.shape}")

# Save the split files
train_set.to_csv("../data/processed/train.csv", index=False)
validation_set.to_csv("../data/processed/validation.csv", index=False)
test_set.to_csv("../data/processed/test.csv", index=False)

2021-07-11 17:24:48,341:INFO:Training shape: (266692, 36)
2021-07-11 17:24:48,343:INFO:Validation shape: (22860, 36)
2021-07-11 17:24:48,344:INFO:Test shape: (91437, 36)


In [7]:
y_train = train_set['price']
y_test = validation_set['price']

# # load the pre-selected features
# # ==============================

X_train = train_set.drop(['price'], axis=1)
X_test = validation_set.drop(['price'], axis=1)

In [8]:
# let's identify the categorical variables
# we will capture those of type object
data = df

cat_vars = [var for var in data.columns if data[var].dtype == 'O']

# MSSubClass is also categorical by definition, despite its numeric values
# (you can find the definitions of the variables in the data_description.txt
# file available on Kaggle, in the same website where you downloaded the data)

# lets add MSSubClass to the list of categorical variables
cat_vars = cat_vars

# cast all variables as categorical
X_train[cat_vars] = X_train[cat_vars].astype('O')
X_test[cat_vars] = X_test[cat_vars].astype('O')

# number of categorical variables
len(cat_vars)

25

In [9]:
# make a list of the categorical variables that contain missing values

cat_vars_with_na = [
    var for var in cat_vars
    if X_train[var].isnull().sum() > 0
]

# print percentage of missing values per variable
X_train[cat_vars_with_na ].isnull().mean().sort_values(ascending=False)

sewage_company       0.996550
secondary_address    0.974664
water_company        0.025096
street               0.004203
primary_address      0.000011
dtype: float64

In [10]:
# variables to impute with the string missing
with_string_missing = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean() > 0.1]

# variables to impute with the most frequent category
with_frequent_category = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean() < 0.1]

In [11]:
# I print the values here, because it makes it easier for
# later when we need to add this values to a config file for 
# deployment

with_string_missing

['secondary_address', 'sewage_company']

In [12]:
with_frequent_category

['primary_address', 'street', 'water_company']

In [13]:
# replace missing values with new label: "Missing"

# set up the class
cat_imputer_missing = CategoricalImputer(
    imputation_method='missing', variables=with_string_missing)

# fit the class to the train set
cat_imputer_missing.fit(X_train)

# the class learns and stores the parameters
cat_imputer_missing.imputer_dict_

{'secondary_address': 'Missing', 'sewage_company': 'Missing'}

In [14]:
# replace NA by missing

# IMPORTANT: note that we could store this class with joblib
X_train = cat_imputer_missing.transform(X_train)
X_test = cat_imputer_missing.transform(X_test)

In [15]:
# replace missing values with most frequent category

# set up the class
cat_imputer_frequent = CategoricalImputer(
    imputation_method='frequent', variables=with_frequent_category)

# fit the class to the train set
cat_imputer_frequent.fit(X_train)

# the class learns and stores the parameters
cat_imputer_frequent.imputer_dict_

{'primary_address': '3',
 'street': 'MAIN STREET',
 'water_company': 'Severn Trent'}

In [16]:
# replace NA by missing

# IMPORTANT: note that we could store this class with joblib
X_train = cat_imputer_frequent.transform(X_train)
X_test = cat_imputer_frequent.transform(X_test)

In [17]:
# check that we have no missing information in the engineered variables

X_train[cat_vars_with_na].isnull().sum()

primary_address      0
secondary_address    0
street               0
water_company        0
sewage_company       0
dtype: int64

In [18]:
# check that test set does not contain null values in the engineered variables

[var for var in cat_vars_with_na if X_test[var].isnull().sum() > 0]

[]

In [19]:
# now let's identify the numerical variables

num_vars = [
    var for var in X_train.columns if var not in cat_vars and var != 'price'
]

# number of numerical variables
len(num_vars)

10

In [20]:
# make a list with the numerical variables that contain missing values
vars_with_na = [
    var for var in num_vars
    if X_train[var].isnull().sum() > 0
]

# print percentage of missing values per variable
X_train[vars_with_na].isnull().mean()

Series([], dtype: float64)

In [21]:
vars_with_na

[]

In [22]:
# add missing indicator

missing_ind = AddMissingIndicator(variables=vars_with_na)

missing_ind.fit(X_train)

X_train = missing_ind.transform(X_train)
X_test = missing_ind.transform(X_test)

# check the binary missing indicator variables
X_train[[]].head()

""
111297
175648
354160
317941
39112


In [23]:
# then replace missing data with the mean

# set the imputer
mean_imputer = MeanMedianImputer(
    imputation_method='mean', variables=vars_with_na)

# learn and store parameters from train set
mean_imputer.fit(X_train)

# the stored parameters
mean_imputer.imputer_dict_

{'latitude': 53.07099148525264,
 'longitude': -1.116544075420335,
 'distance_to_station': 3.0182603011339677,
 'average_income': 42572.84882936121,
 'altitude': 71.79669431404017,
 'adjusted_price': 224765.54705802948,
 'year': 2007.0690609392109,
 'month': 6.804193601607848,
 'quarter': 2.591780030897065}

In [24]:
X_train = mean_imputer.transform(X_train)
X_test = mean_imputer.transform(X_test)

# IMPORTANT: note that we could save the imputers with joblib

# check that we have no more missing values in the engineered variables
X_train[vars_with_na].isnull().sum()

Series([], dtype: float64)

In [25]:
# check that test set does not contain null values in the engineered variables

[var for var in vars_with_na if X_test[var].isnull().sum() > 0]

[]

In [26]:
# now we drop YrSold
drop_features = DropFeatures(features_to_drop=['id', 'date', 'primary_address', 'adjusted_price'])

X_train = drop_features.fit_transform(X_train)
X_test = drop_features.transform(X_test)

In [27]:
X_test.head()

,postcode,type,new_build,land,secondary_address,street,latitude,longitude,grid_ref,county,district,ward,constituency,region,middle_layer_super_output_area,postcode_area,postcode_district,nearest_station,distance_to_station,police_force,water_company,average_income,sewage_company,travel_to_work_area,rural_urban,altitude,region_name,area_code,year,month,quarter
271097,NG19 0GY,D,N,F,Missing,GRIZEDALE CLOSE,53.151406,-1.155928,SK565619,Nottinghamshire,Mansfield,Kingsway,Mansfield,East Midlands,Forest Town & Newlands,NG,NG19,Mansfield Town,3.01566,Nottinghamshire,Severn Trent,38300.0,Missing,Mansfield,Urban city and town,104.0,Mansfield,E07000174,2013,10,4
232432,NG11 6JG,S,N,F,Missing,BARLEYLANDS,52.886951,-1.148915,SK573325,Nottinghamshire,Rushcliffe,Ruddington,Rushcliffe,East Midlands,Ruddington,NG,NG11,Beeston,5.44157,Nottinghamshire,Severn Trent,49300.0,Missing,Nottingham,Urban minor conurbation,38.0,Rushcliffe,E07000176,2009,11,4
132492,NG24 2SU,D,Y,F,Missing,SYERSTON WAY,53.077808,-0.781961,SK816540,Nottinghamshire,Newark and Sherwood,Beacon,Newark,East Midlands,Newark South East,NG,NG24,Newark North Gate,1.27248,Nottinghamshire,Severn Trent,45000.0,Missing,Lincoln,Urban city and town,45.0,Newark and Sherwood,E07000175,2003,2,1
352629,NG17 4EF,T,N,F,Missing,DOWNING STREET,53.130409,-1.256014,SK498595,Nottinghamshire,Ashfield,Central & New Cross,Ashfield,East Midlands,Sutton Forest Side & New Cross,NG,NG17,Sutton Parkway,1.94222,Nottinghamshire,Severn Trent,33200.0,Missing,Mansfield,Urban city and town,155.0,Ashfield,E07000170,2019,3,1
191246,NG23 7HR,D,N,F,Missing,MAIN ROAD,53.175163,-0.764333,SK826649,Nottinghamshire,Newark and Sherwood,Collingham,Newark,East Midlands,"Winthorpe, Coddington & Collingham",NG,NG23,Collingham,3.57600,Nottinghamshire,Anglian Water,49200.0,Anglian Water,Lincoln,Rural village,11.0,Newark and Sherwood,E07000175,2006,8,3


In [28]:
X_train.head()

,postcode,type,new_build,land,secondary_address,street,latitude,longitude,grid_ref,county,district,ward,constituency,region,middle_layer_super_output_area,postcode_area,postcode_district,nearest_station,distance_to_station,police_force,water_company,average_income,sewage_company,travel_to_work_area,rural_urban,altitude,region_name,area_code,year,month,quarter
111297,NG13 8RL,D,N,F,Missing,FOREST ROAD,52.952270,-0.965403,SK696399,Nottinghamshire,Rushcliffe,Bingham West,Newark,East Midlands,Bingham,NG,NG13,Bingham,0.952076,Nottinghamshire,Severn Trent,50600.0,Missing,Nottingham,Rural town and fringe,29.0,Rushcliffe,E07000176,2001,12,4
175648,NG2 7HF,S,N,F,Missing,MICKLEBOROUGH WAY,52.913632,-1.150147,SK572354,Nottinghamshire,Rushcliffe,Compton Acres,Rushcliffe,East Midlands,Compton Acres & Lutterell,NG,NG2,Nottingham,3.724690,Nottinghamshire,Severn Trent,57000.0,Missing,Nottingham,Urban minor conurbation,46.0,Rushcliffe,E07000176,2005,9,3
354160,NG16 1PZ,S,N,F,Missing,HAMMERSMITH CLOSE,52.986667,-1.225937,SK520435,Nottinghamshire,Broxtowe,Nuthall East & Strelley,Broxtowe,East Midlands,"Kimberley South, Trowell & Awsworth",NG,NG16,Bulwell,2.458400,Nottinghamshire,Severn Trent,47100.0,Missing,Nottingham,Urban minor conurbation,91.0,Broxtowe,E07000172,2019,4,2
317941,LE12 6LX,D,N,F,Missing,CASTLE HILL,52.829047,-1.174166,SK557260,Nottinghamshire,Rushcliffe,Leake,Rushcliffe,East Midlands,East Leake,LE,LE12,Loughborough,5.755030,Nottinghamshire,Severn Trent,56800.0,Missing,Nottingham,Rural town and fringe,56.0,Rushcliffe,E07000176,2016,12,4
39112,NG8 3LF,S,N,F,Missing,BEECHDALE ROAD,52.961504,-1.198142,SK539407,Nottinghamshire,Nottingham,Leen Valley,Nottingham South,East Midlands,Bobbers Mill,NG,NG8,Nottingham,3.818050,Nottinghamshire,Severn Trent,33600.0,Missing,Nottingham,Urban minor conurbation,40.0,City of Nottingham,E06000018,1997,7,3


In [31]:
rare_encoder = RareLabelEncoder(tol=0.01, n_categories=3, variables=[
       'postcode', 'secondary_address', 'street','grid_ref',
       'county', 'district', 'ward', 'constituency', 'region',
       'middle_layer_super_output_area', 'postcode_area', 'postcode_district',
       'nearest_station', 'police_force', 'rural_urban', 'region_name', 'area_code',
       'water_company', 'sewage_company', 'travel_to_work_area'])

# find common labels
rare_encoder.fit(X_train)

# the common labels are stored, we can save the class
# and then use it later :)

X_train = rare_encoder.transform(X_train)
X_test = rare_encoder.transform(X_test)

rare_encoder.encoder_dict_

/Users/chris/github/zanalytics/repositories/zanasonic/.venv/lib/python3.9/site-packages/feature_engine/encoding/rare_label.py:179: UserWarning: The number of unique categories for variable region is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


{'postcode': Index([], dtype='object'),
 'secondary_address': Index(['Missing'], dtype='object'),
 'street': Index(['MAIN STREET'], dtype='object'),
 'grid_ref': Index([], dtype='object'),
 'county': Index(['Nottinghamshire'], dtype='object'),
 'district': Index(['Ashfield', 'Rushcliffe', 'Newark and Sherwood', 'Gedling', 'Bassetlaw',
        'Broxtowe', 'Mansfield', 'Nottingham'],
       dtype='object'),
 'ward': Index(['Beacon', 'Hucknall North', 'Worksop North', 'Edwinstowe & Clipstone',
        'Carlton', 'Worksop North West', 'Netherfield', 'Porchester',
        'Toton & Chilwell Meadows', 'Carlton Hill', 'Radcliffe on Trent',
        'Worksop South', 'Hucknall West', 'Leake', 'Hucknall Central'],
       dtype='object'),
 'constituency': Index(['Newark', 'Mansfield', 'Bassetlaw', 'Rushcliffe', 'Gedling', 'Ashfield',
        'Broxtowe', 'Sherwood', 'Nottingham East', 'Nottingham South'],
       dtype='object'),
 'region': array(['East Midlands', 'Yorkshire and The Humber'], dtype=o

In [36]:
# set up the encoder
one_hot_encoder = OneHotEncoder(top_categories=None, variables=['type', 'new_build', 'land'], drop_last=True)

# fit the encoder
encoder.fit(X_train)

# transform the data
# train_t= encoder.transform(X_train)
# test_t= encoder.transform(X_test)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)
encoder.encoder_dict_


{'type': ['D', 'S', 'T', 'F'], 'new_build': ['N'], 'land': ['F', 'L']}

In [33]:
# set up the encoder
cat_encoder = OrdinalEncoder(encoding_method='ordered', variables=[
       'postcode', 'secondary_address', 'street','grid_ref',
       'county', 'district', 'ward', 'constituency', 'region',
       'middle_layer_super_output_area', 'postcode_area', 'postcode_district',
       'nearest_station', 'police_force', 'rural_urban', 'region_name', 'area_code',
       'water_company', 'sewage_company', 'travel_to_work_area'])

# create the mappings
cat_encoder.fit(X_train, y_train)

X_train = cat_encoder.transform(X_train)
X_test = cat_encoder.transform(X_test)

# mappings are stored and class can be saved
cat_encoder.encoder_dict_

{'postcode': {'Rare': 0},
 'secondary_address': {'Rare': 0, 'Missing': 1},
 'street': {'Rare': 0, 'MAIN STREET': 1},
 'grid_ref': {'Rare': 0},
 'county': {'Nottinghamshire': 0, 'Rare': 1},
 'district': {'Nottingham': 0,
  'Mansfield': 1,
  'Ashfield': 2,
  'Bassetlaw': 3,
  'Broxtowe': 4,
  'Gedling': 5,
  'Newark and Sherwood': 6,
  'Rare': 7,
  'Rushcliffe': 8},
 'ward': {'Netherfield': 0,
  'Carlton Hill': 1,
  'Worksop North West': 2,
  'Hucknall Central': 3,
  'Carlton': 4,
  'Hucknall North': 5,
  'Worksop North': 6,
  'Beacon': 7,
  'Hucknall West': 8,
  'Worksop South': 9,
  'Rare': 10,
  'Edwinstowe & Clipstone': 11,
  'Porchester': 12,
  'Toton & Chilwell Meadows': 13,
  'Radcliffe on Trent': 14,
  'Leake': 15},
 'constituency': {'Rare': 0,
  'Nottingham East': 1,
  'Nottingham South': 2,
  'Ashfield': 3,
  'Mansfield': 4,
  'Bassetlaw': 5,
  'Gedling': 6,
  'Sherwood': 7,
  'Broxtowe': 8,
  'Newark': 9,
  'Rushcliffe': 10},
 'region': {'East Midlands': 0, 'Yorkshire and The 

In [37]:
X_train.head(2)

,postcode,secondary_address,street,latitude,longitude,grid_ref,county,district,ward,constituency,region,middle_layer_super_output_area,postcode_area,postcode_district,nearest_station,distance_to_station,police_force,water_company,average_income,sewage_company,travel_to_work_area,rural_urban,altitude,region_name,area_code,year,month,quarter,type_D,type_S,type_T,type_F,new_build_N,land_F,land_L
111297,0,1,0,52.952270,-0.965403,0,0,8,10,9,0,31,2,18,19,0.952076,0,0,50600.0,0,3,2,29.0,8,8,2001,12,4,1,0,0,0,1,1,0
175648,0,1,0,52.913632,-1.150147,0,0,8,10,10,0,38,2,19,20,3.724690,0,0,57000.0,0,3,1,46.0,8,8,2005,9,3,0,1,0,0,1,1,0


In [38]:
# create scaler
scaler = MinMaxScaler()

#  fit  the scaler to the train set
scaler.fit(X_train) 

# transform the train and test set

# sklearn returns numpy arrays, so we wrap the
# array with a pandas dataframe

X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns
)

X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_train.columns
)

In [39]:
X_train.head()

,postcode,secondary_address,street,latitude,longitude,grid_ref,county,district,ward,constituency,region,middle_layer_super_output_area,postcode_area,postcode_district,nearest_station,distance_to_station,police_force,water_company,average_income,sewage_company,travel_to_work_area,rural_urban,altitude,region_name,area_code,year,month,quarter,type_D,type_S,type_T,type_F,new_build_N,land_F,land_L
0,0.0,1.0,0.0,0.231717,0.603192,0.0,0.0,1.0,0.666667,0.9,0.0,0.794872,0.5,0.750000,0.791667,0.067063,0.0,0.0,0.603175,0.0,0.75,0.50,0.134328,1.0,1.0,0.230769,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,1.0,0.0,0.175254,0.348066,0.0,0.0,1.0,0.666667,1.0,0.0,0.974359,0.5,0.791667,0.833333,0.263607,0.0,0.0,0.748299,0.0,0.75,0.25,0.218905,1.0,1.0,0.384615,0.727273,0.666667,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.281983,0.243403,0.0,0.0,0.5,0.666667,0.8,0.0,0.589744,0.5,0.416667,0.333333,0.173843,0.0,0.0,0.523810,0.0,0.75,0.25,0.442786,0.5,0.5,0.923077,0.272727,0.333333,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,0.0,1.0,0.0,0.051646,0.314897,0.0,0.0,1.0,1.000000,1.0,0.0,0.564103,1.0,0.958333,0.958333,0.407532,0.0,0.0,0.743764,0.0,0.75,0.50,0.268657,1.0,1.0,0.807692,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,1.0,0.0,0.245212,0.281787,0.0,0.0,0.0,0.666667,0.2,0.0,0.564103,0.5,0.166667,0.833333,0.270225,0.0,0.0,0.217687,0.0,0.75,0.25,0.189055,0.0,0.0,0.076923,0.545455,0.666667,0.0,1.0,0.0,0.0,1.0,1.0,0.0
